# Wordle

In [ ]:
import jupyter_black

import pathlib
import string
from typing import Optional, TypedDict

jupyter_black.load(lab=False, line_length=79)

## Build the lookup tables

Read possible guesses (from https://gist.github.com/cfreshman).

In [ ]:
words = pathlib.Path("data/wordle-nyt-words-14855.txt").read_text().split()
assert all(len(word) == 5 for word in words)
assert len(set(words)) == len(words)
assert all(word.islower() for word in words)

Iterate over words and populate lookups

In [ ]:
contains: dict[str, set[str]] = {}
does_not_contain: dict[str, set[str]] = {}
contains_at: dict[tuple[str, int], set[str]] = {}

for letter in string.ascii_lowercase:
    contains[letter] = set()
    does_not_contain[letter] = set()
    for position in range(5):
        contains_at[(letter, position)] = set()

alphabet = set(string.ascii_lowercase)

for word in words:
    for position, letter in enumerate(word):
        contains[letter].add(word)
        contains_at[(letter, position)].add(word)
    for letter in alphabet - set(word):
        does_not_contain[letter].add(word)

contains_not_at: dict[tuple[str, int], set[str]] = {}

for letter in string.ascii_lowercase:
    for position in range(5):
        contains_not_at[(letter, position)] = (
            contains[letter] - contains_at[(letter, position)]
        )

for lookup in [contains, does_not_contain, contains_at, contains_not_at]:
    assert set.union(*(lookup.values())) == set(words)

In [ ]:
class Lookup(TypedDict):
    contains_at: dict[tuple[str, int], set[str]]
    does_not_contain: dict[str, set[str]]
    contains_not_at: dict[tuple[str, int], set[str]]


lookup: Lookup = {
    "contains_at": contains_at,
    "does_not_contain": does_not_contain,
    "contains_not_at": contains_not_at,
}

Function to return candidate solutions given a hint

In [ ]:
def candidates(
    hint: str,
    incorrect_positions: Optional[set[int]] = None,
    lookup: Lookup = lookup,
) -> set[str]:
    """Return possible Wordle solutions given a hint.

    hint is a 5-letter word specification. Use uppercase letters
    to indicate letters in the correct position (green) and
    lowercase letters to indicate letters that are either in the
    incorrect position (orange) or do not exist in the word
    (dark grey). Use incorrect_positions to indicate the indices
    in the hint for the letters that exist in the word but are
    incorrectly positioned (i.e., color letters orange using
    these indices).

    For example, hint = 'Tseta' and incorrect_positions = set([1, 3])
    specify that the word starts with a T, contains s and another
    t but in incorrect positions and does not contain an a.
    """

    if incorrect_positions is None:
        incorrect_positions = set()

    if len(hint) != 5:
        raise ValueError(f"hint {hint} must be of length 5")
    if len(incorrect_positions) > 5:
        raise ValueError(
            f"length of incorrect_positions {incorrect_positions} must "
            "be at most 5"
        )
    for incorrect_position in incorrect_positions:
        if not (0 <= incorrect_position < 5):
            raise ValueError(
                f"incorrect position {incorrect_position} out of bounds"
            )

    word_sets = []
    for position, letter in enumerate(hint):
        if letter.isupper():
            if position in incorrect_positions:
                raise ValueError(
                    "cannot have overlap between correct "
                    "letters and incorrect_positions"
                )
            word_sets.append(lookup["contains_at"][(letter.lower(), position)])
        elif position in incorrect_positions:
            word_sets.append(lookup["contains_not_at"][(letter, position)])
        else:
            word_sets.append(lookup["does_not_contain"][letter])
    return set.intersection(*word_sets)

In [ ]:
assert candidates("HELPS") == set(["helps"])

In [ ]:
assert candidates("HEplS", incorrect_positions=[2, 3]) == set(["helps"])

In [ ]:
candidates("praps", incorrect_positions=[0, 1, 2, 3, 4])